# Analyze TIES Hyperparameter Gridsearch

## Imports

In [1]:
import os
import json
import pandas as pd

# Iterate Through Each Hyperparameter Combination folder

In [3]:
unlearn_method = "ca"
training_method = "independent"
merge_method = "ties"
concept_type = "style"
concepts = ["Winter"]
OUTPUT_ROOT = os.environ.get("OUTPUT_ROOT", "/CUIG_Results")
base_dir = f"{OUTPUT_ROOT}/{unlearn_method}/eval_results/{training_method}/merge/{merge_method}/{concept_type}"
all_results = []

# Find hypeparameter values
print(f"Searching in {base_dir}")
combos = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
print(f"Found {len(combos)} combinations in {base_dir}")

# Iterate through each combination
for combo in combos:
    combo_dir = os.path.join(base_dir, combo)
    for concept in concepts:
        # Each concept folder contains a "metrics" folder with the JSON files
        metrics_dir = os.path.join(combo_dir, f"thru{concept}", 'metrics')
        summary_file = os.path.join(metrics_dir, 'summary.json')
        
        if os.path.exists(summary_file):
            with open(summary_file, 'r') as f:
                summary_data = json.load(f)

            unlearn_accuracy_avg = summary_data.get('UA', {})
            retention_accuracy = summary_data.get('IRA', {})
            cross_retention_accuracy = summary_data.get('CRA', {})
            final_score = (unlearn_accuracy_avg + (retention_accuracy + cross_retention_accuracy)/2) / 2
            
            all_results.append({
                'combo': combo,
                'concept': concept,
                'concept_type': concept_type,
                'UA': unlearn_accuracy_avg,
                'IRA': retention_accuracy,
                'CRA': cross_retention_accuracy,
                'final_score': final_score
            })
        else:
            print(f"Missing metrics for combo {combo} and concept {concept} in {combo}")

# Combine results into a DataFrame for further analysis
df_results = pd.DataFrame(all_results)

# ---------------------------
# 2. Extract Hyperparameter Values from 'combo'
# ---------------------------
# The combo string is in the format "2Rank1Alpha0.1Dropout"
if merge_method == "ties":
    pattern = r'lambda(?P<lambda>[\d\.]+)_topk(?P<topk>[\d\.]+)'
extracted = df_results['combo'].str.extract(pattern)
df_results['topk'] = extracted['topk'].astype(float)
df_results['lambda'] = extracted['lambda'].astype(float)

df_results.sort_values(['final_score', 'UA'], ascending=False, inplace=True)

REPO_ROOT = os.environ.get("REPO_ROOT", "/CUIG")
df_save_dir = f"{REPO_ROOT}/Analysis/Notebooks/notebook_output/{unlearn_method}/{training_method}/merge/{merge_method}/{concept_type}"
os.makedirs(df_save_dir, exist_ok=True)
df_results.to_excel(os.path.join(df_save_dir, f"thru{concepts[0]}.xlsx"), index=False)
print(f"Results saved to {df_save_dir}/thru{concepts[0]}.xlsx")

print(f"Total number of combinations: {len(df_results)}")
df_results.head(len(df_results))

Searching in /fs/scratch/PAS2099/lee.10369/CUIG/ca/eval_results/independent/merge/ties/style
Found 20 combinations in /fs/scratch/PAS2099/lee.10369/CUIG/ca/eval_results/independent/merge/ties/style
Missing metrics for combo lambda2.00_topk0.40 and concept Winter in lambda2.00_topk0.40
Missing metrics for combo lambda2.00_topk0.20 and concept Winter in lambda2.00_topk0.20
Missing metrics for combo lambda2.00_topk0.80 and concept Winter in lambda2.00_topk0.80
Missing metrics for combo lambda2.00_topk0.60 and concept Winter in lambda2.00_topk0.60
Missing metrics for combo lambda2.50_topk0.20 and concept Winter in lambda2.50_topk0.20
Results saved to /users/PAS2099/justinhylee135/Research/UnlearningDM/CUIG/Analysis/Notebooks/notebook_output/ca/independent/merge/ties/style/thruWinter.xlsx
Total number of combinations: 15


,combo,concept,concept_type,UA,IRA,CRA,final_score,topk,lambda
5,lambda2.50_topk0.40,Winter,style,95.45,51.46,80.65,80.7525,0.4,2.50
13,lambda2.50_topk0.60,Winter,style,88.41,52.08,86.96,78.9650,0.6,2.50
10,lambda2.50_topk0.80,Winter,style,82.27,55.21,92.06,77.9525,0.8,2.50
3,lambda1.75_topk0.20,Winter,style,71.36,58.33,88.05,72.2750,0.2,1.75
0,lambda1.75_topk0.40,Winter,style,63.86,63.54,94.24,71.3750,0.4,1.75
2,lambda1.75_topk0.60,Winter,style,50.91,64.38,96.08,65.5700,0.6,1.75
4,lambda1.50_topk0.20,Winter,style,48.64,66.04,91.74,63.7650,0.2,1.50
7,lambda1.50_topk0.40,Winter,style,40.91,66.88,95.87,61.1425,0.4,1.50
9,lambda1.75_topk0.80,Winter,style,38.18,70.62,96.30,60.8200,0.8,1.75
8,lambda1.50_topk0.60,Winter,style,31.59,73.96,96.52,58.4150,0.6,1.50
